In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import timeit
import tracemalloc
undersample = RandomUnderSampler(sampling_strategy='majority')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense
from tensorflow.keras.layers import GRU, LSTM
from attention import Attention
from tensorflow.keras.callbacks import LearningRateScheduler

In [2]:
import glob
file_name_Despress = glob.glob('Depresjon\condition\*.csv')
file_name_Normal = glob.glob('Depresjon\control\*.csv')

In [3]:
def readfile(file_name):
    ID = 0
    result = []
    for f in file_name:
        df = pd.read_csv(f)
        out = [x[11:] for x in df.timestamp]df.timestamp = out     
        df_Pivot = pd.pivot_table(df,index=["timestamp"],columns=["date"],fill_value=0)
        if (df_Pivot.shape[-1]>7):
            for j in range(7, df_Pivot.shape[-1]+1):
                result.append(df_Pivot.iloc[:, j-7:j])
    return np.array(result)

In [4]:
df_Depress = readfile(file_name_Despress)
df_Nomal = readfile(file_name_Normal)

In [5]:
print(df_Depress.shape)
print(df_Nomal.shape)

(267, 1440, 7)
(547, 1440, 7)


In [6]:
df_Final = np.concatenate((df_Depress, df_Nomal))

In [7]:
label =[]
for i in range(0, df_Depress.shape[0]+df_Nomal.shape[0]):
    if i <df_Depress.shape[0]:
        label.append(1)
    else:
        label.append(0)

In [8]:
df_Final.shape

(814, 1440, 7)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_Final, label, test_size=0.2, random_state=42)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*7)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*7)

In [11]:
from sklearn.metrics import confusion_matrix
    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

# Using DL

In [12]:
from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical

# one hot encode
y_train = to_categorical(y_train)
# one hot encode
y_test = to_categorical(y_test)

In [13]:
# This is a sample of a scheduler I used in the past
def lr_scheduler(epoch, lr):
    if epoch % 50 == 0 and epoch:
        lr = lr - lr*0.1
        return lr
    return lr

In [14]:
def training(X, y, iteration, epoch):
    #X = X.reshape((X.shape[0], X.shape[1], 1))
    
    print(X.shape)
    print(y.shape)
    time_steps = X.shape[1]
    input_dim = 1
    print((time_steps, input_dim))
    
    model_path = 'DL_model_Bagging/model_'+str(iteration)+'.h5'
    # define the keras model
    model_input = Input(shape=(time_steps, input_dim))
    x = GRU(128, return_sequences=True)(model_input)
    x = Attention(64)(x)
    x = Dense(32, activation = "relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(2, activation = "sigmoid")(x)
    model = Model(model_input, x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #print(model.summary())
    
    # train.
    model.fit(X, y, epochs=epoch, batch_size=160, validation_split=0.2,verbose=1, 
                           callbacks = [LearningRateScheduler(lr_scheduler, verbose=0),
                                        tensorflow.keras.callbacks.ModelCheckpoint(model_path,
                                                                                   monitor='val_loss',
                                                                                   save_best_only=True,
                                                                                   mode='min',
                                                                                   verbose=0)])

In [15]:
from tensorflow.keras.models import load_model

def model_Predict(X):
    RM_predicted =[]
    for i in range(0, num_Interation):
        model = load_model('DL_model_Bagging/model_'+str(i)+".h5") 
        RM_predicted.append(model.predict(X))
    return RM_predicted

In [16]:
from sklearn.metrics import confusion_matrix
    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

In [19]:
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
import timeit
import tracemalloc

# define undersample strategy

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
# split 10 fold of undersampling
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from tensorflow.keras import layers, Model

max_auc = 0.7
for num_Interation in range(20, 50, 5):
    rsFile = "Result_OverIteration_Bagging_Numeric/NumberOfBag_"+str(num_Interation)+".csv"
    f2=open(rsFile,"w")
    f2.write('num_Interation, epoch, Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    for epoch in (50, 100, 500):

        print("WORKING ON ITERATION "+str(num_Interation)+" epoch: ", str(epoch))
        
        
        
        X_train_List=[]
        y_train_List = []
        for i in range(0, num_Interation):
            X_over, y_over = undersample.fit_resample(X_train, np.argmax(y_train, axis=1))
            X_train_List.append(X_over)

            y_over = to_categorical(y_over)
            y_train_List.append(y_over)

        X_train_List = np.array(X_train_List)
        y_train_List = np.array(y_train_List)

        X_train_List = np.array(X_train_List)
        y_train_List = np.array(y_train_List)
        
        
        # ============== Starting training
        start = timeit.default_timer()
        tracemalloc.start()
        
        ## Start training
        for i in range(0, num_Interation):
            #X_train_List[i] = X_train_List[i].reshape(X_train_List[i].shape[0], X_train_List[i].shape[1], 1)
            training(X_train_List[i].reshape(X_train_List[i].shape[0], X_train_List[i].shape[1], 1), y_train_List[i] , i, epoch)
        
        current, peak = tracemalloc.get_traced_memory()
        stop = timeit.default_timer()
        Time_train=stop - start
        Train_Peak_RAM = peak / 10**6
        
        
        start = timeit.default_timer()
        tracemalloc.start()

        predicted = model_Predict(X_test)

        stop = timeit.default_timer()
        Time_predict=stop - start
        #thres = predicted.min()
        current, peak = tracemalloc.get_traced_memory()
        Peak_Time_prediction = peak / 10**6



        for i in range(0, num_Interation):
            result = np.array([[0]*predicted[i].shape[1]]*predicted[i].shape[0])

        for i in range(0, num_Interation):
            #print(result)
            result = result+ predicted[i]
            ##print('*=================')



        #print('result: ', result)

        auc=roc_auc_score(np.argmax(y_test, axis = 1), result[:, 1]/num_Interation)
        

        if max_auc <= auc:
            #max_auc = auc
            print('===================: ')
            print('======= AUC ==============: ', auc)
            print('=====AUC===AUC=====: ')
            print('===AUC========AUC==: ')
            print('=AUC===========AUC=: ')
            print('=AUC===========AUC=: ')
            print('=AUC===========AUC=: ')
            print('===================: ')
            print('===================: ')



        thres = result.min()
        while (thres <= result.max()):
        #for thres in range(1, 1001):
            thres = thres+(result.max()/300)
            yhat = []
            for i in range(len(result)):
                if result[i][0]>thres:
                    yhat.append(0)
                else:
                    yhat.append(1)

            yhat = np.array(yhat)

            tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(np.argmax(y_test, axis = 1), yhat)
            #auc = roc_plot(y_test, yhat)
            f2.write(str(num_Interation)+", "+str(epoch)+", "+str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")


    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       